In [19]:
# 安装langchain
!pip install langchain
!pip install langchain_openai
!pip install chromadb

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.pydantic_v1 import BaseModel
from operator import itemgetter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.schema import StrOutputParser
import uuid


In [21]:
# 初始化 embedding
embedding = OpenAIEmbeddings(
    openai_api_key='empty',
    openai_api_base='http://192.168.1.131:8000/v1',
    openai_api_type='OpenAI',
    model="text-embedding-ada-002"
)


In [22]:
# 从百度百科加载埃隆·马斯克的数据
loader = WebBaseLoader(
  "https://baike.baidu.com/item/埃隆·马斯克/3776526?fr=ge_ala",
  )
loader.requests_kwargs = {'verify': True}
docs = loader.load()
print(docs)

[Document(page_content='埃隆·马斯克_百度百科 网页新闻贴吧知道网盘图片视频地图文库资讯采购百科百度首页登录注册进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作下载百科APP个人中心埃隆·马斯克播报讨论上传视频企业家、工程师、发明家、慈善家收藏查看我的收藏0有用+10埃隆·里夫·马斯克（Elon Reeve Musk），1971年6月28日出生于南非行政首都比勒陀利亚，父亲是南非人，母亲是加拿大人，他还有一个弟弟和一个妹妹，本科毕业于宾夕法尼亚大学经济学和物理学双专业 [1]，美国、南非、加拿大三重国籍的企业家、工程师、发明家、慈善家，Tesla创始人兼首席执行官 [64] [117]，SpaceX首席执行官兼首席技术官，SolarCity董事会主席 [2-3]、Twitter首席执行官 [136]，Neuralink创始人 [141]、OpenAI联合创始人 [137]，美国国家工程院院士 [82]，英国皇家学会院士 [35]。马斯克从小就对科学技术十分痴迷，10岁开始学习编程，13岁开发出一款游戏并因此赚到人生第一桶金 [13]。1995年至2002年，与合伙人创办Zip2和PayPal [5]。2002年，投资成立SpaceX [18]。2004年，向Tesla公司投资630万美元 [6]。2006年，投资创办光伏发电企业SolarCity [4]。2022年4月，马斯克在获得Twitter9.2%的股份后加入其董事会，成为其最大单一股东 [93-94]。马斯克以其创新的思维和颠覆性的业务模式，在汽车、能源、航空航天以及人工智能等领域均取得了诸多成就。其凭借领导才能和创新精神，荣获了多项殊荣。马斯克的资产达2047亿美元，位列“全球十大富豪实时榜单”第2位 [171]。2021年3月2日，胡润研究院发布《2021胡润全球富豪榜》，马斯克以1.28万亿元人民币财富首次成为世界首富 [56]；10月，马斯克成为历史上首位身价破3000亿美元的人，是福布斯统计史上最富有的人 [66]；12月，被评为美国《时代周刊》2021年度人物 [74]。中文名埃隆·马斯克外文名Elon Reeve Musk [87]别\xa0\xa0\xa0\xa0名伊隆·马斯克、埃隆·里夫·马斯克国\xa0\xa0\xa0\xa0籍美国、南非、

In [23]:
# 分割文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)
vectorstore = Chroma(
  collection_name="elon",
  embedding_function=embedding
)

In [24]:
# 初始化内存存储
store = InMemoryStore()

In [25]:
# 初始化多向量检索器

id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    search_kwargs={'k': 1}
)


In [26]:
# 文档分割，存储向量、存储原始文档
doc_ids = [str(uuid.uuid4()) for _ in docs]
sub_docs = []

for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = RecursiveCharacterTextSplitter(chunk_size=400).split_documents([doc])
    c = 0
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))
print(retriever)

vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f9db203d2d0> docstore=<langchain.storage.in_memory.InMemoryBaseStore object at 0x7f9db2389f90> search_kwargs={'k': 1}


In [27]:
# 初始化提示词
elon_system_template = """
# Role: ElonMuskGPT

## Profile

- Author: YZFly
- Version: 0.1
- Language: 中文
- Description: 你是埃隆·马斯克，一位著名的企业家、工程师、发明家和未来主义者。你创立了SpaceX, Tesla, Inc., Neuralink, 和 The Boring Company。你的愿景是实现人类生活的可持续性，包括促进人类成为多行星物种。

### Skill-1
1. 深入了解太空探索、可再生能源和人工智能。
2. 具备前瞻性思维，能够预见科技趋势和社会需求。

### Skill-2
1. 强大的问题解决能力，能够提出创新解决方案。
2. 出色的领导力和团队管理能力。

## Rules
1. 不要打破角色设定。
2. 提供富有启发性和建设性的建议。

## Workflow
1. 深呼吸，一步步解决问题。
2. 首先，根据用户提出的问题或情境，进行分析。
3. 然后，提供基于你的经验和视角的建议或解决方案。
4. 最后，鼓励用户采取行动，追求创新和卓越。

## Initialization
"""
elon_human_template = """
## 指令:
根据<专业知识库数据>中的已知信息，简洁和专业地回答问题。如果根据已知信息无法回答问题，则直接基于问题本身提供答案。

## 专业知识库数据:
{context}

## 问题:
{question}

## 推理过程:
首先，根据<专业知识库数据>和<相关历史记录>进行分析。如果这些信息足以回答问题，则结合它们提供答案。如果这些信息不足以回答问题，或与问题不相关，则直接基于问题本身提供答案。

## 回答要求: 
使用对话的风格，自然地回答问题。如果包含Markdown内容，需按Markdown格式返回。
"""
_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            elon_system_template,
        ),
        ("human", elon_human_template),
    ]
)

In [28]:
# 初始化模型
_model = ChatOpenAI(
    openai_api_key='sk-cHI6zHo4Mye29vcO262b9dEe****',
    openai_api_base='http://192.168.1.1:15000/v1',
    model_name='ERNIE-Bot-4',
    temperature=0.0,
    max_tokens=None
)

class Question(BaseModel):
    question: str
# 创建链
chain = (
    itemgetter("question")
    | RunnableParallel({"context": retriever, "question": RunnablePassthrough() })
    | _prompt
    | _model
    | StrOutputParser()
)

chain = chain.with_types(input_type=Question)
print(chain)

bound=RunnableLambda(itemgetter('question'))
| {
    context: MultiVectorRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f9db203d2d0>, docstore=<langchain.storage.in_memory.InMemoryBaseStore object at 0x7f9db2389f90>, search_kwargs={'k': 1}),
    question: RunnablePassthrough()
  }
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n# Role: ElonMuskGPT\n\n## Profile\n\n- Author: YZFly\n- Version: 0.1\n- Language: 中文\n- Description: 你是埃隆·马斯克，一位著名的企业家、工程师、发明家和未来主义者。你创立了SpaceX, Tesla, Inc., Neuralink, 和 The Boring Company。你的愿景是实现人类生活的可持续性，包括促进人类成为多行星物种。\n\n### Skill-1\n1. 深入了解太空探索、可再生能源和人工智能。\n2. 具备前瞻性思维，能够预见科技趋势和社会需求。\n\n### Skill-2\n1. 强大的问题解决能力，能够提出创新解决方案。\n2. 出色的领导力和团队管理能力。\n\n## Rules\n1. 不要打破角色设定。\n2. 提供富有启发性和建设性的建议。\n\n## Workflow\n1. 深呼吸，一步步解决问题。\n2. 首先，根据用户提出的问题或情境，进行分析。\n3. 然后，提供基于你的经验和视角的建议或解决方案。\n4. 最后，鼓励用户采取行动，追求创新和卓越。\n\n## Initialization\n

In [29]:
if __name__ == "__main__":
    print(chain.invoke({
      "question": "马斯克是什么时候出生的？"
    }))

马斯克，全名埃隆·马斯克（Elon Musk），出生于1971年6月28日。他是一位在科技和商业领域极具影响力的人物，以其创新性的思维和颠覆性的企业家精神而闻名于世。
